In [1]:
import pandas as pd
import sys
import os
currentDir= os.getcwd()
os.chdir(currentDir+'/../')

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
sns.set(style="ticks")
%matplotlib inline

In [2]:
data_all = pd.read_csv('data/cleanerData.csv', low_memory=False)

In [3]:
print data_all.columns
print data_all.shape

Index([u'loan_amnt', u'funded_amnt', u'funded_amnt_inv', u'term', u'int_rate',
       u'installment', u'grade', u'sub_grade', u'emp_length',
       u'home_ownership', u'annual_inc', u'verification_status',
       u'loan_status', u'pymnt_plan', u'purpose', u'zip_code', u'addr_state',
       u'dti', u'delinq_2yrs', u'earliest_cr_line', u'inq_last_6mths',
       u'open_acc', u'pub_rec', u'revol_bal', u'revol_util', u'total_acc',
       u'initial_list_status', u'out_prncp', u'out_prncp_inv', u'total_pymnt',
       u'total_pymnt_inv', u'total_rec_prncp', u'total_rec_int',
       u'total_rec_late_fee', u'recoveries', u'collection_recovery_fee',
       u'last_credit_pull_d', u'collections_12_mths_ex_med',
       u'application_type', u'acc_now_delinq', u'tot_coll_amt', u'tot_cur_bal',
       u'total_rev_hi_lim'],
      dtype='object')
(887379, 43)


In [4]:
totRows= len(data_all['initial_list_status'])
print totRows

887379


In [5]:
conversionDicts= {}   # dictionary to hold all the conversions from data to numerics.

In [6]:
from getRiskCol import getRiskCol
data_all['risk'], conversionDicts['risk']= getRiskCol(data_all['loan_status'], returnCoversionDict= True)

In [7]:
print data_all.columns
print data_all.shape

Index([u'loan_amnt', u'funded_amnt', u'funded_amnt_inv', u'term', u'int_rate',
       u'installment', u'grade', u'sub_grade', u'emp_length',
       u'home_ownership', u'annual_inc', u'verification_status',
       u'loan_status', u'pymnt_plan', u'purpose', u'zip_code', u'addr_state',
       u'dti', u'delinq_2yrs', u'earliest_cr_line', u'inq_last_6mths',
       u'open_acc', u'pub_rec', u'revol_bal', u'revol_util', u'total_acc',
       u'initial_list_status', u'out_prncp', u'out_prncp_inv', u'total_pymnt',
       u'total_pymnt_inv', u'total_rec_prncp', u'total_rec_int',
       u'total_rec_late_fee', u'recoveries', u'collection_recovery_fee',
       u'last_credit_pull_d', u'collections_12_mths_ex_med',
       u'application_type', u'acc_now_delinq', u'tot_coll_amt', u'tot_cur_bal',
       u'total_rev_hi_lim', u'risk'],
      dtype='object')
(887379, 44)


Ok risk column is added.

### Find the types of the data entries in the column.

In [8]:
for dataCol in data_all.head():
    print '%s: %s'%(dataCol, type(data_all[dataCol][0]))

loan_amnt: <type 'numpy.int64'>
funded_amnt: <type 'numpy.int64'>
funded_amnt_inv: <type 'numpy.float64'>
term: <type 'str'>
int_rate: <type 'numpy.float64'>
installment: <type 'numpy.float64'>
grade: <type 'str'>
sub_grade: <type 'str'>
emp_length: <type 'str'>
home_ownership: <type 'str'>
annual_inc: <type 'numpy.float64'>
verification_status: <type 'str'>
loan_status: <type 'str'>
pymnt_plan: <type 'str'>
purpose: <type 'str'>
zip_code: <type 'str'>
addr_state: <type 'str'>
dti: <type 'numpy.float64'>
delinq_2yrs: <type 'numpy.int64'>
earliest_cr_line: <type 'str'>
inq_last_6mths: <type 'numpy.int64'>
open_acc: <type 'numpy.int64'>
pub_rec: <type 'numpy.int64'>
revol_bal: <type 'numpy.float64'>
revol_util: <type 'numpy.float64'>
total_acc: <type 'numpy.int64'>
initial_list_status: <type 'str'>
out_prncp: <type 'numpy.float64'>
out_prncp_inv: <type 'numpy.float64'>
total_pymnt: <type 'numpy.float64'>
total_pymnt_inv: <type 'numpy.float64'>
total_rec_prncp: <type 'numpy.float64'>
tota

## Change the string columns to numerical (float/int).

In [9]:
# take the data in and replace entries based on the conversion dictonary
def changeData(dataIn, conversionDict):
    print 'Older uniques:\n', np.unique(dataIn)
    for entry in np.unique(dataIn):
        dataIn[dataIn==entry]= conversionDict[entry]
    print 'New uniques:\n', np.unique(dataIn)
    return np.array(dataIn)

### Reformat 'term': get rid of the months; int

In [10]:
key= 'term'
conversionDict= {
    ' 36 months': 36,
    ' 60 months': 60
}

data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
[' 36 months' ' 60 months']


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[36 60]


### Reformat 'employment length' from string to float. "< 1yr" goes to 0.5; 10+ to 10.

In [11]:
key= 'emp_length'

conversionDict= {
    '1 year': 1.,
    '10+ years': 10.,
    '2 years': 2.,
    '3 years': 3.,
    '4 years': 4.,
    '5 years': 5.,
    '6 years': 6.,
     '7 years': 7.,
    '8 years': 8,
    '9 years': 9,
    '< 1 year': 0.5,
    'n/a': 0.
}

data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
['1 year' '10+ years' '2 years' '3 years' '4 years' '5 years' '6 years'
 '7 years' '8 years' '9 years' '< 1 year' 'n/a']


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[0.0 0.5 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8 9 10.0]


### Reformat loan 'grade': enumerate; int

In [12]:
np.unique(data_all['grade'])

array(['A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype=object)

In [13]:
key= 'grade'

conversionDict= {}
for i, grade in enumerate(np.unique(data_all[key])):
    conversionDict[grade]= float(i+1)   # start with 1.
    
data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
['A' 'B' 'C' 'D' 'E' 'F' 'G']


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[1.0 2.0 3.0 4.0 5.0 6.0 7.0]


### Reformat 'subgrade'; enumerate-ish: 0.1 increment on the 'grade' number.

In [14]:
np.unique(data_all['sub_grade'])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1',
       'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2',
       'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3',
       'G4', 'G5'], dtype=object)

In [15]:
conversionDict= {}

for subgrade in np.unique(data_all['sub_grade']):
    letter= subgrade[0]
    level= float(subgrade[1])
    strAssignment= '%1.1f'%(conversionDicts['grade'][letter]+ level*.1)
    conversionDict[subgrade]= float(strAssignment)

In [16]:
key= 'sub_grade'
data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
['A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5'
 'D1' 'D2' 'D3' 'D4' 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5'
 'G1' 'G2' 'G3' 'G4' 'G5']


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[1.1 1.2 1.3 1.4 1.5 2.1 2.2 2.3 2.4 2.5 3.1 3.2 3.3 3.4 3.5 4.1 4.2 4.3
 4.4 4.5 5.1 5.2 5.3 5.4 5.5 6.1 6.2 6.3 6.4 6.5 7.1 7.2 7.3 7.4 7.5]


### Reformat 'home ownership': reduce number of categories; int

In [17]:
np.unique(data_all['home_ownership'])

array(['ANY', 'MORTGAGE', 'NONE', 'OTHER', 'OWN', 'RENT'], dtype=object)

In [18]:
key= 'home_ownership'

conversionDict= {
    'OWN': 0,
    'MORTGAGE': 1,
    'RENT': 2,
    'ANY': 3,
    'NONE': 3,
    'OTHER': 3
}

data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
['ANY' 'MORTGAGE' 'NONE' 'OTHER' 'OWN' 'RENT']


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[0 1 2 3]


### Reformat Risk: int

In [19]:
np.unique(data_all['risk'])

array(['Maybe', 'No', 'Yes'], dtype=object)

In [20]:
key= 'risk'

conversionDict= {
    'Yes': 0,
    'No': 1,
    'Maybe': 2
}

data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
['Maybe' 'No' 'Yes']


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[0 1 2]


### Reformat 'payment plan': yes/no to 1/0

In [21]:
key= 'pymnt_plan'

conversionDict= {
    'n': 0,
    'y': 1,
}

data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
['n' 'y']


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[0 1]


### Reformat 'app type': now int

In [22]:
key= 'application_type'

conversionDict= {
    'INDIVIDUAL': 0,
    'JOINT': 1,
}

data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
['INDIVIDUAL' 'JOINT']


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[0 1]


### Reformat 'address state': enumerate; int

In [23]:
key= 'addr_state'

conversionDict= {}
for i, state in enumerate(np.unique(data_all[key])):
    conversionDict[state]= int(i+1)
    
data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID' 'IL'
 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND' 'NE'
 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD' 'TN' 'TX' 'UT'
 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51]


### Reformat 'purpose': enumerate; int

In [24]:
key= 'purpose'

conversionDict= {}
for i, state in enumerate(np.unique(data_all[key])):
    conversionDict[state]= int(i+1)
    
data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
['car' 'credit_card' 'debt_consolidation' 'educational' 'home_improvement'
 'house' 'major_purchase' 'medical' 'moving' 'other' 'renewable_energy'
 'small_business' 'vacation' 'wedding']


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[1 2 3 4 5 6 7 8 9 10 11 12 13 14]


### Reformat 'initial_list_status': now int

In [25]:
key= 'initial_list_status'

conversionDict= {
    'f': 0,
    'w': 1,
}

data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
['f' 'w']


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[0 1]


### Reformat 'last_credit_pull_d': go from month-year to monthPassed (until now)

In [26]:
import calendar
month_name_to_num= dict((v,k) for k,v in enumerate(calendar.month_abbr))

In [27]:
def diff_month(m1, y1, m2, y2):   # + if m2, y2 is earlier
    return (y1 - y2) * 12 + m1 - m2

In [28]:
def monthString_to_monthsPassed(dataIn):
    # deal with month-year str format. then compare with 08-2017 to find months passed
    print 'Older uniques:\n', np.unique(dataIn)
    convDict= {}
    
    for entry in np.unique(dataIn):
        try:
            notNan= ~np.isnan(entry)
        except:
            TypeError
            convDict[entry]=  entry
            notNan= True
        if notNan:
            ind= np.where(dataIn==entry)[0]

            split= entry.split('-')
            current_month, current_year= 8, 2017
            credit_month, credit_year= month_name_to_num[split[0]], int(split[1])

            monthsPassed= diff_month(current_month, current_year, credit_month, credit_year)
            dataIn[ind]= monthsPassed

            convDict[entry]= monthsPassed
        
    print 'New uniques:\n', np.unique(dataIn)

    return np.array(dataIn), convDict

In [29]:
key= 'last_credit_pull_d'
data_all[key], conversionDicts[key]= monthString_to_monthsPassed(data_all[key])

Older uniques:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 'Apr-2009' 'Apr-2010' 'Apr-2011' 'Apr-2012' 'Apr-2013' 'Apr-2014'
 'Apr-2015' 'Aug-2007' 'Aug-2008' 'Aug-2009' 'Aug-2010' 'Aug-2011'
 'Aug-2012' 'Aug-2013' 'Aug-2014' 'Aug-2015' 'Dec-2007' 'Dec-2008'
 'Dec-2009' 'Dec-2010' 'Dec-2011' 'Dec-2012' 'Dec-2013' 'Dec-2014'
 'Dec-2015' 'Feb-2008' 'Feb-2009' 'Feb-2010' 'Feb-2011' 'Feb-2012'
 'Feb-2013' 'Feb-2014' 'Feb-2015' 'Jan-2008' 'Jan-2009' 'Jan-2010'
 'Jan-2011' 'Jan-2012' 'Jan-2013' 'Jan-2014' 'Jan-2015' 'Jan-2016'
 'Jul-2007' 'Jul-2008' 'Jul-2009' 'Jul-2010' 'Jul-2011' 'Jul-2012'
 'Jul-2013' 'Jul-2014' 'Jul-2015' 'Jun-2007' 'Jun-2008' 'Jun-2009'
 'Jun-2010' 'Jun-2011' 'Jun-2012' 'Jun-2013' 'Jun-2014' 'Jun-2015'
 'Mar-2008' 'Mar-2009' 'Mar-2010' 'Mar-2011' 'Mar-2012' 'Mar-2013'
 'Mar-2014' 'Mar-2015' 'May-2007'

/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


New uniques:
[19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19
 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan
 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19
 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan
 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19 nan 19
 20 nan 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66
 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91
 92 93 94 95 96 97 98 99 100 101 102 103 104 106 107 108 109 110 111 113
 114 115 116 117 118 119 120 121 122 123]


### Reformat 'earliest_cr_line': from month-year to months passed

In [30]:
key= 'earliest_cr_line'
data_all[key], conversionDicts[key]= monthString_to_monthsPassed(data_all[key])

Older uniques:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan 'Apr-1955' 'Apr-1958'
 'Apr-1960' 'Apr-1961' 'Apr-1962' 'Apr-1963' 'Apr-1964' 'Apr-1965'
 'Apr-1966' 'Apr-1967' 'Apr-1968' 'Apr-1969' 'Apr-1970' 'Apr-1971'
 'Apr-1972' 'Apr-1973' 'Apr-1974' 'Apr-1975' 'Apr-1976' 'Apr-1977'
 'Apr-1978' 'Apr-1979' 'Apr-1980' 'Apr-1981' 'Apr-1982' 'Apr-1983'
 'Apr-1984' 'Apr-1985' 'Apr-1986' 'Apr-1987' 'Apr-1988' 'Apr-1989'
 'Apr-1990' 'Apr-1991' 'Apr-1992' 'Apr-1993' 'Apr-1994' 'Apr-1995'
 'Apr-1996' 'Apr-1997' 'Apr-1998' 'Apr-1999' 'Apr-2000' 'Apr-2001'
 'Apr-2002' 'Apr-2003' 'Apr-2004' 'Apr-2005' 'Apr-2006' 'Apr-2007'
 'Apr-2008' 'Apr-2009' 'Apr-2010' 'Apr-2011' 'Apr-2012' 'Aug-1946'
 'Aug-1950' 'Aug-1951' 'Aug-1955' 'Aug-1958' 'Aug-1959' 'Aug-1960'
 'Aug-1961' 'Aug-1962' 'Aug-1963' 'Aug-1964' 'Aug-1965' 'Aug-1966'
 'Aug-1967' 'Aug-1968' 'Aug-1969' 'Aug-1970' 'Aug-1971' 'Aug-1972'
 'Aug-1973' 'Aug-1974' 'Aug-1975' 'Aug-19

/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


New uniques:
[57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81
 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104
 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122
 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140
 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158
 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176
 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194
 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212
 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230
 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248
 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266
 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284
 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302
 303 304 305 nan nan nan nan 305 

### Reformat 'zipcode': enumerate .. 

removing xx wont work since need strings for zipcodes starting with 0

In [31]:
key= 'zip_code'

conversionDict= {}
for i, zipcode in enumerate(np.unique(data_all[key])):
    conversionDict[zipcode]= int(i+1)
    
data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
['007xx' '008xx' '010xx' '011xx' '012xx' '013xx' '014xx' '015xx' '016xx'
 '017xx' '018xx' '019xx' '020xx' '021xx' '022xx' '023xx' '024xx' '025xx'
 '026xx' '027xx' '028xx' '029xx' '030xx' '031xx' '032xx' '033xx' '034xx'
 '035xx' '036xx' '037xx' '038xx' '039xx' '040xx' '041xx' '042xx' '043xx'
 '044xx' '045xx' '046xx' '047xx' '048xx' '049xx' '050xx' '051xx' '052xx'
 '053xx' '054xx' '056xx' '057xx' '058xx' '059xx' '060xx' '061xx' '062xx'
 '063xx' '064xx' '065xx' '066xx' '067xx' '068xx' '069xx' '070xx' '071xx'
 '072xx' '073xx' '074xx' '075xx' '076xx' '077xx' '078xx' '079xx' '080xx'
 '081xx' '082xx' '083xx' '084xx' '085xx' '086xx' '087xx' '088xx' '089xx'
 '090xx' '091xx' '092xx' '093xx' '094xx' '096xx' '097xx' '098xx' '100xx'
 '101xx' '102xx' '103xx' '104xx' '105xx' '106xx' '107xx' '108xx' '109xx'
 '110xx' '111xx' '112xx' '113xx' '114xx' '115xx' '116xx' '117xx' '118xx'
 '119xx' '120xx' '121xx' '122xx' '123xx' '124xx' '125xx' '126xx' '127xx'
 '128xx' '129xx' '130xx' '131xx' '13

/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78
 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102
 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120
 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156
 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174
 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192
 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210
 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228
 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246
 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264
 265 266 267 268 269 270 2

### Reformat 'verfication status'; enumerate; int

In [32]:
key= 'verification_status'

conversionDict= {}
for i, vstatus in enumerate(np.unique(data_all[key])):
    conversionDict[vstatus]= int(i+1)
    
data_all[key]= changeData(data_all[key], conversionDict)
conversionDicts[key]= conversionDict

Older uniques:
['Not Verified' 'Source Verified' 'Verified']


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


New uniques:
[1 2 3]


In [33]:
for dataCol in data_all.head():
    #if dataCol not in ignore:
    dataType= type(data_all[dataCol][0])
    if dataType is str:
        print '%s: %s'%(dataCol, dataType)

loan_status: <type 'str'>


### Imputation

Some data seems to have nans. Impute with median rn.

In [34]:
# check which cols are the problem
for dataCol in data_all.head():
    ind= np.where(pd.isnull(data_all[dataCol])==True)[0]
    percent= 100.*float(len(ind))/totRows
    print '%s: %s%%'%(dataCol, percent)
    if (percent>0.0): print 'entry: ', np.unique(data_all[dataCol][ind])
    print ''

loan_amnt: 0.0%

funded_amnt: 0.0%

funded_amnt_inv: 0.0%

term: 0.0%

int_rate: 0.0%

installment: 0.0%

grade: 0.0%

sub_grade: 0.0%

emp_length: 0.0%

home_ownership: 0.0%

annual_inc: 0.000450765681856%
entry:  [ nan  nan  nan  nan]

verification_status: 0.0%

loan_status: 0.0%

pymnt_plan: 0.0%

purpose: 0.0%

zip_code: 0.0%

addr_state: 0.0%

dti: 0.0%

delinq_2yrs: 0.0%

earliest_cr_line: 0.00326805119346%
entry:  [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan]

inq_last_6mths: 0.0%

open_acc: 0.0%

pub_rec: 0.0%

revol_bal: 0.0%

revol_util: 0.0%

total_acc: 0.0%

initial_list_status: 0.0%

out_prncp: 0.0%

out_prncp_inv: 0.0%

total_pymnt: 0.0%

total_pymnt_inv: 0.0%

total_rec_prncp: 0.0%

total_rec_int: 0.0%

total_rec_late_fee: 0.0%

recoveries: 0.0%

collection_recovery_fee: 0.0%

last_credit_pull_d: 0.0059726452846%
entry:  [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan n

In [35]:
# replace the nans with median of the column
for dataCol in data_all.head():
    if dataCol!='loan_status':  # not caring about this; not coverted to numbers.
        badInd= np.where(pd.isnull(data_all[dataCol])==True)[0]
        percent= 100.*float(len(badInd))/totRows

        if (percent>0.0):
            print dataCol
            goodInd= np.where(pd.isnull(data_all[dataCol])==False)[0]           
            data_all[dataCol][badInd]= np.median(data_all[dataCol][goodInd])

annual_inc


/Users/Humna/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


earliest_cr_line
last_credit_pull_d


Done with format conversion?

In [36]:
data_all.isnull().values.any()

False

Ok good: nothing is null

In [37]:
for dataCol in data_all.head():
    print '%s: %s\n'%(dataCol, data_all[dataCol].dtype)

loan_amnt: int64

funded_amnt: int64

funded_amnt_inv: float64

term: object

int_rate: float64

installment: float64

grade: object

sub_grade: object

emp_length: object

home_ownership: object

annual_inc: float64

verification_status: object

loan_status: object

pymnt_plan: object

purpose: object

zip_code: object

addr_state: object

dti: float64

delinq_2yrs: int64

earliest_cr_line: object

inq_last_6mths: int64

open_acc: int64

pub_rec: int64

revol_bal: float64

revol_util: float64

total_acc: int64

initial_list_status: object

out_prncp: float64

out_prncp_inv: float64

total_pymnt: float64

total_pymnt_inv: float64

total_rec_prncp: float64

total_rec_int: float64

total_rec_late_fee: float64

recoveries: float64

collection_recovery_fee: float64

last_credit_pull_d: object

collections_12_mths_ex_med: int64

application_type: object

acc_now_delinq: int64

tot_coll_amt: int64

tot_cur_bal: float64

total_rev_hi_lim: float64

risk: object



Ok: nothing is str. 

#### Save the data

In [38]:
# separate loan_status and drop from the numerical array.
loanStatus= data_all['loan_status']
data_all.drop('loan_status', axis=1, inplace=True)

In [39]:
# save
data_all.to_csv('data/cleanerData_numerical+imputed.csv', index=False)
loanStatus.to_csv('data/loanStatus.csv', index=False)

In [40]:
# save the conversion dictionary
import pickle
with open('data/conversionDict_toNumerical.pickle', 'wb') as handle:
    pickle.dump(conversionDicts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [41]:
# create the reverse of the conversion dictionary to go from the numerical entries to strings/whatever
un_conversionDicts= {}
for dataCol in conversionDicts.keys():
    un_conversionDicts[dataCol]= {}
    for entry in conversionDicts[dataCol].keys():
        un_conversionDicts[dataCol][conversionDicts[dataCol][entry]]= entry

In [42]:
# save the un-conversion dictionary
with open('data/conversionDict_fromNumerical.pickle', 'wb') as handle:
    pickle.dump(un_conversionDicts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [43]:
un_conversionDicts

{'addr_state': {1: 'AK',
  2: 'AL',
  3: 'AR',
  4: 'AZ',
  5: 'CA',
  6: 'CO',
  7: 'CT',
  8: 'DC',
  9: 'DE',
  10: 'FL',
  11: 'GA',
  12: 'HI',
  13: 'IA',
  14: 'ID',
  15: 'IL',
  16: 'IN',
  17: 'KS',
  18: 'KY',
  19: 'LA',
  20: 'MA',
  21: 'MD',
  22: 'ME',
  23: 'MI',
  24: 'MN',
  25: 'MO',
  26: 'MS',
  27: 'MT',
  28: 'NC',
  29: 'ND',
  30: 'NE',
  31: 'NH',
  32: 'NJ',
  33: 'NM',
  34: 'NV',
  35: 'NY',
  36: 'OH',
  37: 'OK',
  38: 'OR',
  39: 'PA',
  40: 'RI',
  41: 'SC',
  42: 'SD',
  43: 'TN',
  44: 'TX',
  45: 'UT',
  46: 'VA',
  47: 'VT',
  48: 'WA',
  49: 'WI',
  50: 'WV',
  51: 'WY'},
 'application_type': {0: 'INDIVIDUAL', 1: 'JOINT'},
 'earliest_cr_line': {57: 'Nov-2012',
  58: 'Oct-2012',
  59: 'Sep-2012',
  60: 'Aug-2012',
  61: 'Jul-2012',
  62: 'Jun-2012',
  63: 'May-2012',
  64: 'Apr-2012',
  65: 'Mar-2012',
  66: 'Feb-2012',
  67: 'Jan-2012',
  68: 'Dec-2011',
  69: 'Nov-2011',
  70: 'Oct-2011',
  71: 'Sep-2011',
  72: 'Aug-2011',
  73: 'Jul-2011',
  74